In [1]:
import numpy as np
import tensorflow as tf
import random 
import os

np.random.seed(17)
tf.random.set_seed(17)
random.seed(17)

In [2]:
densenet = tf.keras.applications.densenet.DenseNet201()

In [3]:
cat_img = tf.keras.preprocessing.image.load_img('cat.jpg', target_size=(224,224))

In [4]:
cat_img.show()

In [5]:
cat_img = tf.keras.preprocessing.image.img_to_array(cat_img)

In [6]:
cat_img.shape

(224, 224, 3)

In [7]:
cat_img = np.expand_dims(cat_img, axis=0)
cat_img = tf.keras.applications.densenet.preprocess_input(cat_img)
cat_img

array([[[[1.0330508, 1.7633053, 2.5528543],
         [0.9988013, 1.7282913, 2.5179958],
         [0.9988013, 1.7282913, 2.5179958],
         ...,
         [1.2042983, 1.8333333, 2.5702832],
         [1.2042983, 1.8333333, 2.5702832],
         [1.2042983, 1.8333333, 2.5702832]],

        [[1.015926 , 1.7457983, 2.535425 ],
         [0.9988013, 1.7282913, 2.5179958],
         [0.9988013, 1.7282913, 2.5179958],
         ...,
         [1.2042983, 1.8333333, 2.5702832],
         [1.2042983, 1.8333333, 2.5702832],
         [1.2042983, 1.8333333, 2.5702832]],

        [[1.015926 , 1.7457983, 2.535425 ],
         [0.9988013, 1.7282913, 2.5179958],
         [0.9988013, 1.7282913, 2.5179958],
         ...,
         [1.2042983, 1.8333333, 2.5702832],
         [1.2042983, 1.8333333, 2.5702832],
         [1.2042983, 1.8333333, 2.5702832]],

        ...,

        [[1.6837914, 2.2009804, 2.6051416],
         [1.7180408, 2.2009804, 2.622571 ],
         [1.7180408, 2.2009804, 2.622571 ],
         ...,


In [8]:
pred = densenet.predict(cat_img)

2023-04-03 23:54:54.522050: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 2s 2s/step


In [9]:
tf.keras.applications.densenet.decode_predictions(pred)

[[('n02124075', 'Egyptian_cat', 0.1692347),
  ('n01882714', 'koala', 0.14802781),
  ('n01883070', 'wombat', 0.11369825),
  ('n02108915', 'French_bulldog', 0.109237164),
  ('n02125311', 'cougar', 0.045628116)]]

In [10]:
for layer in densenet.layers:
    layer.trainable = False
    
for layer in densenet.layers[-10:]:
    layer.trainable = True

densenet.layers[-1].activation = tf.keras.activations.relu

In [11]:
model_cats = tf.keras.models.Sequential([
    densenet,
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [12]:
model_cats.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet201 (Functional)    (None, 1000)              20242984  
                                                                 
 dense (Dense)               (None, 1)                 1001      
                                                                 
Total params: 20,243,985
Trainable params: 2,204,625
Non-trainable params: 18,039,360
_________________________________________________________________


In [13]:
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()

def f1_mettrics(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2 * ((prec * rec)/(prec + rec + 1e-7))

In [27]:
model_cats.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.binary_crossentropy,
    metrics=[f1_mettrics]
)

In [15]:
def preprocess_image(file):
    img = tf.keras.preprocessing.image.load_img(file, target_size=(224,224))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.applications.densenet.preprocess_input(img)
    return img

In [16]:
cats = [(preprocess_image('pics/cats/'+file), 1) for file in os.listdir('pics/cats')]
        
nocats = [(preprocess_image('pics/nocats/'+file), 0) for file in os.listdir('pics/nocats')]

In [17]:
all_pics = cats + nocats

In [18]:
random.shuffle(all_pics)

In [19]:
x = np.array([a[0] for a in all_pics])
y = np.array([a[1] for a in all_pics])

In [20]:
def train_val_test(x, val_frac=.15, test_frac=.15):
    x_train = x[:round((1-val_frac-test_frac) * len(x))]
    x_val = x[round((1-val_frac-test_frac)*len(x)):round((1-test_frac)*len(x))]
    x_test = x[round((1-test_frac) * len(x)):]
    
    return x_train, x_val, x_test

x_train, x_val, x_test = train_val_test(x)
y_train, y_val, y_test = train_val_test(y)

In [21]:
x_train.shape, x_val.shape, x_test.shape

((321, 224, 224, 3), (68, 224, 224, 3), (69, 224, 224, 3))

In [22]:
y_train.shape, y_val.shape, y_test.shape

((321,), (68,), (69,))

In [26]:
model_cats.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    batch_size=16,
    epochs=50
)

Epoch 1/50


ValueError: in user code:

    File "/Users/evgenijgrinev/.pyenv/versions/3.10.10/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Users/evgenijgrinev/.pyenv/versions/3.10.10/lib/python3.10/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/evgenijgrinev/.pyenv/versions/3.10.10/lib/python3.10/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Users/evgenijgrinev/.pyenv/versions/3.10.10/lib/python3.10/site-packages/keras/engine/training.py", line 891, in train_step
        self._validate_target_and_loss(y, loss)
    File "/Users/evgenijgrinev/.pyenv/versions/3.10.10/lib/python3.10/site-packages/keras/engine/training.py", line 857, in _validate_target_and_loss
        raise ValueError(

    ValueError: No loss found. 

In [24]:
round(0.9675,1)

1.0